In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import time
import torch
from datetime import datetime, timedelta
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
import re
from six import BytesIO
import matplotlib.dates as mdates
import scipy.stats as stats
import seaborn as sns
import pymysql.cursors
import spacy
import ftfy
import ast
import math
import itertools
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
def get_previous_date(date_str, days_before):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    before_date = date_obj - timedelta(days=days_before)
    before_date_str = before_date.strftime('%Y/%#m/%#d')
    return before_date_str

def get_above_date(date_str, days_after):
    date_obj = datetime.strptime(date_str, '%Y/%m/%d')
    after_date = date_obj + timedelta(days=days_after)
    after_date_str = after_date.strftime('%Y/%#m/%#d')
    return after_date_str

get_above_date('2015/12/30', 3)
get_previous_date('2015/12/10', 3)

# Load data from the Hugging Face

Daily data for stock

In [ ]:
#get data from each parquet file

#file1
df = pd.read_parquet("train-00000-of-00004.parquet")
df.to_csv("stockdata1.csv", index=False)
us_1 = df['symbol'].unique()

#file2
df2 = pd.read_parquet("train-00001-of-00004.parquet")
df2.to_csv("stockdata2.csv", index=False)
us_2 = df2['symbol'].unique()

#file3
df3 = pd.read_parquet("train-00002-of-00004.parquet")
df3.to_csv("stockdata3.csv", index=False)
us_3 = df3['symbol'].unique()

#file4
df4 = pd.read_parquet("train-00003-of-00004.parquet")
df4.to_csv("stockdata4.csv", index=False)
us_4 = df4['symbol'].unique()

Get the list of stock name

In [ ]:
#merge 4 parquet
array1 = np.array(us_1)
array2 = np.array(us_2)
array3 = np.array(us_3)
array4 = np.array(us_4)

stock_list = np.concatenate([array1, array2, array3, array4])
stock_list

Balance sheet for stocks (to calculate turnover rate)

In [ ]:
df5 = pd.read_parquet("stock_sheet.parquet")
df5.to_csv("balance_sheet.csv", index=False)

Delete the data before 2010 & Only leave symbol, date, close and volume

In [ ]:
# for daily price files
df["date"] = pd.to_datetime(df["date"])
df1_new = df[(df["date"] >= "2010-01-01") & (df['date']<= "2024-12-31")][["symbol", "date", "volume", "adj_close"]]

df2["date"] = pd.to_datetime(df2["date"])
df2_new = df2[(df2["date"] >= "2010-01-01") & (df2['date']<= "2024-12-31")][["symbol", "date", "volume", "adj_close"]]

df3["date"] = pd.to_datetime(df3["date"])
df3_new = df3[(df3["date"] >= "2010-01-01") & (df3['date']<= "2024-12-31")][["symbol", "date", "volume", "adj_close"]]

df4["date"] = pd.to_datetime(df4["date"])
df4_new = df4[(df4["date"] >= "2010-01-01") & (df4['date']<= "2024-12-31")][["symbol", "date", "volume", "adj_close"]]

#merge all the data
df_stockdata = pd.concat([df1_new, df2_new, df3_new, df4_new], ignore_index=True)
df_stockdata.to_csv('stocklist.csv', index=False)

In [ ]:
df_stockdata = pd.read_csv('stocklist.csv')
df_stockdata

In [ ]:
# for balance sheet data
df5["date"] = pd.to_datetime(df5["date"])
dfshares = df5[(df5["date"] >= "2010-01-01") & (df5['date']<= "2024-12-31")][["symbol", "date", "common_stock_shares_outstanding"]]

#save to csv
dfshares.to_csv("bs2.csv", index=False)

Leave the stocks which are both included in two dataset

In [ ]:
common_symbols = set(dfshares["symbol"]).intersection(set(df_stockdata["symbol"]))
dfshares = dfshares[dfshares["symbol"].isin(common_symbols)]
df_stockdata = df_stockdata[df_stockdata["symbol"].isin(common_symbols)]

#check whether the length of the financial data and the trading data are the same
len(dfshares['symbol'].unique()) == len(df_stockdata['symbol'].unique())

Delete the stocks do not have data before 2023/4 (according to the setting of the experiment)

In [ ]:
df_stockdata["date"] = pd.to_datetime(df_stockdata["date"])

# the setting time
cutoff_date = pd.Timestamp("2023-04-01")

# the earlist trading day for each stock
first_trade_dates = df_stockdata.groupby("symbol")["date"].min()

#only save stocks had traded before 2023/04
valid_symbols = first_trade_dates[first_trade_dates < cutoff_date].index

# save stocks in all sets
df_stockdata = df_stockdata[df_stockdata["symbol"].isin(valid_symbols)]
dfshares = dfshares[dfshares["symbol"].isin(valid_symbols)]
len(dfshares['symbol'].unique()) == len(df_stockdata['symbol'].unique())

Get timeline (align with balance sheet one)

In [ ]:
df_timeline = df_stockdata['date'].unique()
df_timeline

Get the list of the stocks whehter have transaction exception

In [ ]:
#all trading dates
all_trading_dates = set(df_timeline)

# not satify stocks
stocks_with_missing_dates = []

for symbol, group in df_stockdata.groupby("symbol"):
    stock_dates = set(group["date"])
    
    #the first trade date of the stock
    first_trading_day = min(stock_dates)
    last_trading_day = max(stock_dates)
    
    # the whole tading dates that the stock should have
    required_dates = {d for d in all_trading_dates if first_trading_day <= d <= last_trading_day}
    
    # check whether miss any
    if stock_dates != required_dates:
        stocks_with_missing_dates.append(symbol)

#the abnormal stock
print("These stocks cannot cover all the trading days：")
print(stocks_with_missing_dates)

In [ ]:
df_stockdata = df_stockdata[~df_stockdata["symbol"].isin(stocks_with_missing_dates)]
dfshares = dfshares[~dfshares["symbol"].isin(stocks_with_missing_dates)]
len(dfshares['symbol'].unique()) == len(df_stockdata['symbol'].unique())

Merge data for each month

In [ ]:
df_stockdata['date'] = pd.to_datetime(df_stockdata['date'])

# add year-month
df_stockdata['year_month'] = df_stockdata['date'].dt.to_period('M')

def summarize_month(group):
    volume_sum = group['volume'].sum()
    first_price = group.iloc[0]['adj_close']
    last_price = group.iloc[-1]['adj_close']
    adj_close_change = (last_price - first_price) / first_price if first_price != 0 else None
    return pd.Series({'volume': volume_sum, 'adj_close_change': adj_close_change})

df_stockdata_monthly = df_stockdata.sort_values('date').groupby(['symbol', 'year_month']).apply(summarize_month).reset_index()

df_stockdata_monthly

In [ ]:
#save to the csv
df_stockdata_monthly.to_csv('stocklist_monthly.csv', index=False)

Merge df_stockdata_monthly and dfshares

In [ ]:
# merge directly, not consider NaN
dfshares['date'] = pd.to_datetime(dfshares['date'])
dfshares['year_month'] = dfshares['date'].dt.to_period('M').astype(str)
df_stockdata_monthly['year_month'] = df_stockdata_monthly['year_month'].astype(str)

# create a dict
shares_lookup = {
    (row.symbol, row.year_month): row.common_stock_shares_outstanding
    for row in dfshares.itertuples(index=False)
}

# add by apply
df_stockdata_monthly['common_stock_shares_outstanding'] = df_stockdata_monthly.apply(
    lambda row: shares_lookup.get((row.symbol, row.year_month), None),
    axis=1
)

#fill the table
# fill to head
df_stockdata_monthly['common_stock_shares_outstanding'] = (
    df_stockdata_monthly.groupby('symbol')['common_stock_shares_outstanding']
    .ffill()
)

# fill to tail
df_stockdata_monthly['common_stock_shares_outstanding'] = (
    df_stockdata_monthly.groupby('symbol')['common_stock_shares_outstanding']
    .bfill()
)

df_stockdata_monthly

In [ ]:
#get the final df_stockdata_monthly
df_stockdata_monthly['turnover_rate'] = (
    df_stockdata_monthly['volume'] / df_stockdata_monthly['common_stock_shares_outstanding']
)

df_stockdata_monthly['turnover_rate'] = df_stockdata_monthly['turnover_rate'].round(4)

# delete rows where turnover_rate is NaN
df_stockdata_monthly = df_stockdata_monthly.dropna(subset=['turnover_rate'])

#delete NaN of the shares
df_stockdata_monthly = df_stockdata_monthly.dropna(subset=['common_stock_shares_outstanding'])

df_stockdata_monthly['year'] = df_stockdata_monthly['year_month'].str[:4].astype(int)

# count how many data before 2020 (for training)
symbol_pre2020_counts = (
    df_stockdata_monthly[df_stockdata_monthly['year'] < 2020]
    .groupby('symbol')
    .size()
)

#find the symbol that includes less 20 data 
symbols_to_filter_out = symbol_pre2020_counts[symbol_pre2020_counts < 20].index.tolist()

# delete the data
df_stockdata_monthly = df_stockdata_monthly[
    ~((df_stockdata_monthly['symbol'].isin(symbols_to_filter_out)) & (df_stockdata_monthly['year'] < 2020))
].reset_index(drop=True)

# delete the "year" colume
df_stockdata_monthly.drop(columns=['year'], inplace=True)

df_stockdata_monthly

#save to the csv
df_stockdata_monthly.to_csv("stocklist_monthly.csv", index=False)

# Building the training set

In [ ]:
df_pivot_ready = df_stockdata_monthly[['symbol', 'year_month', 'adj_close_change', 'turnover_rate']].copy()

# align with the df
df_pivot_ready = df_pivot_ready.rename(columns={
    'adj_close_change': 'profit',
    'turnover_rate': 'turnover' 
})

# multi-index，and transform unstack to wide format
df_wide = df_pivot_ready.set_index(['symbol', 'year_month']).unstack('year_month')

df_wide.columns = [ (ym, metric) for metric, ym in df_wide.columns ]

# rank index
df_wide = df_wide.sort_index(axis=1, level=0)

df_input_monthly = df_wide

#save to the file
df_input_monthly.to_csv("stock_input.csv")

df_input_monthly

In [ ]:
#let "symbol" in the df be one of the columns in the df (it was the index at the beginning)
df_input_monthy = df_input_monthly.reset_index()

In [ ]:
def find_symbols_with_nan(df, late_date, n):
    #obtain the first 2n colume
    columns_to_check = df.columns[df.columns.get_loc(late_date) - 2 * n : df.columns.get_loc(late_date)]

    # find the column including NaN and return the symbol of the colume
    symbols_with_nan = df[df[columns_to_check].isna().any(axis=1)].index

    return set(symbols_with_nan)

In [ ]:
def create_df_profit_rate_with_symbol(df_input, date_str, date):  # date is the index of the date
    
    #find the symbol including NaN in the past 40 col
    symbols_with_nan = find_symbols_with_nan(df_input, date_str, 40)
    
    # save the symbol excluded in the NaN set
    df_profit_rate = df_input.loc[~df_input.index.isin(symbols_with_nan)]

    #the past 40 col + present col (41 col in addition)
    selected_columns = list(df_profit_rate.columns[date - 40 : date + 1])

    df_profit_rate = df_profit_rate[selected_columns]
    df_profit_rate = df_profit_rate.reset_index()

    #sort and remove NaN
    df_profit_rate = df_profit_rate.sort_values(by=[(date_str, 'profit')], ascending=False).dropna().reset_index(drop=True)

    return df_profit_rate


In [ ]:
def create_df_profit_rate(df_input, date_str, date):  # date is the index of the date
    
    #find the symbol including NaN in the past 40 col
    symbols_with_nan = find_symbols_with_nan(df_input, date_str, 40)
    
    # save symbol without NaN
    df_profit_rate = df_input.loc[~df_input.index.isin(symbols_with_nan)]
    
    # choose the 40 col + present 1 col
    df_profit_rate = df_profit_rate[df_profit_rate.columns[date - 40:date + 1]]

    # sort by (date_str, 'profit')
    df_profit_rate = df_profit_rate.sort_values(by=[date_str], ascending=False)

    # remove prossible NaN
    df_profit_rate = df_profit_rate.dropna().reset_index(drop=True)

    return df_profit_rate


In [ ]:
def normalize_dataframe(df):
    #calculate the mean and std for each column
    mean_values = df.iloc[:, 0:-1].mean()
    std_values = df.iloc[:, 0:-1].std()

    # normalize except the last colume (for traget label)
    for col in df.columns[0:-1]:
        df[col] = (df[col] - mean_values[col]) / std_values[col]

    return df

In [ ]:
#the first date of the dataset is the 40th col
date = 40
output_list = []
input_list = []
#the end of the training dataset is 12/2019, which is the 239th col:
while date < 239:

    #get the next date
    date_str = df_input_monthy.columns[date]

    # check whether there are NaN. If included, skip this
    symbols_with_nan = find_symbols_with_nan(df_input_monthy, date_str, 20)
   
    df_profit_rate = create_df_profit_rate(df_input_monthy, date_str, date)

    #targte label
    #date_str_profit_col = f'{date_str}_profit'
        # categrory the stocks based on the t+1 profit with the setting quantile (20%)
    quantiles = pd.qcut(df_profit_rate[date_str], q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=False)
    df_profit_rate['quantiles'] = quantiles
    
        # the quantile to the traget label: [1, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1]
    encoding = {0: [0, 0, 1], 1: [0, 0, 0], 2: [0, 1, 0], 3: [0, 0, 0], 4: [1, 0, 0]}
    df_profit_rate['encoding'] = df_profit_rate['quantiles'].map(encoding)
    
        #save these to the list
    output_list_temp = df_profit_rate['encoding'].tolist()
    output_list = output_list + output_list_temp
    
    
        #z-score normarlize
    df_profit_nor = normalize_dataframe(df_profit_rate)
    
        # get the first 40th col before the target date that everydate contains a vector, combine them
    profit_turnoff_cols = [col for col in df_profit_nor.columns if 'profit' in col or 'turnoff' in col]
    normalized_df = df_profit_nor.iloc[:, :40]
        
    vector_list_per_row = normalized_df.apply(lambda row: [list(row[i:i+2]) for i in range(0, len(row), 2)], axis=1)
        
        #save to the input_daily.txt
    input_list_temp = vector_list_per_row.tolist()
    input_list = input_list + input_list_temp
    
    date = date + 2
    print(date_str)
                  
#save this to the output.txt
file_path = 'train_output_monthy.txt'

with open(file_path, 'w') as file:
    for item in output_list:
            file.write(str(item) + '\n')        
        
#save to input.txt    
# 指定要写入的文本文件路径
file_path = 'train_input_monthy.txt'

with open(file_path, 'w') as file:
    for item in input_list:
            file.write(str(item) + '\n')
            

# Training

Quantformer

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.W_Q = nn.Linear(d_model, d_model)
        self.W_K = nn.Linear(d_model, d_model)
        self.W_V = nn.Linear(d_model, d_model)
        self.W_O = nn.Linear(d_model, d_model)
 
    def forward(self, Q, K, V):
        Q = self.W_Q(Q)
        K = self.W_K(K)
        V = self.W_V(V)

        Q = self._split_heads(Q)
        K = self._split_heads(K)
        V = self._split_heads(V)

        attention_weights = torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.depth, dtype=torch.float32))
        attention_weights = torch.softmax(attention_weights, dim=-1)

        output = torch.matmul(attention_weights, V)
        output = self._combine_heads(output)

        output = self.W_O(output)
        return output
 
    def _split_heads(self, tensor):
        tensor = tensor.view(tensor.size(0), -1, self.num_heads, self.depth)
        return tensor.transpose(1, 2)
 
    def _combine_heads(self, tensor):
        tensor = tensor.transpose(1, 2).contiguous()
        return tensor.view(tensor.size(0), -1, self.num_heads * self.depth)


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
 
    def forward(self, x):
        attention_output = self.attention(x, x, x)
        attention_output = self.norm1(x + attention_output)

        feedforward_output = self.feedforward(attention_output)
        output = self.norm2(attention_output + feedforward_output)
        return output


class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.encoder_attention = MultiHeadAttention(d_model, num_heads)
        self.feedforward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
 
    def forward(self, x, encoder_output):
        self_attention_output = self.self_attention(x, x, x)
        self_attention_output = self.norm1(x + self_attention_output)

        encoder_attention_output = self.encoder_attention(self_attention_output, encoder_output, encoder_output)
        encoder_attention_output = self.norm2(self_attention_output + encoder_attention_output)

        feedforward_output = self.feedforward(encoder_attention_output)
        output = self.norm3(encoder_attention_output + feedforward_output)
        return output

class Quantformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, output_dim):
        super(Quantformer, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.encoder_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_heads) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(hidden_dim, num_heads) for _ in range(num_layers)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)
 
    def forward(self, x):
        # Input layer
        x = self.input_layer(x)

        # Encoder layers
        encoder_output = x.transpose(0, 1)
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output)

        # Decoder layers
        decoder_output = encoder_output
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output, encoder_output)

        # Select the output of the last time step
        decoder_output = decoder_output[-1, :, :]

        # Output layer
        output = self.output_layer(decoder_output)
        return output


model = Quantformer(input_dim=2, hidden_dim=64, num_heads=8, num_layers=6, output_dim = 3)

In [ ]:
def read_data(input_file, output_file, num_samples):
    with open(input_file, 'r') as f_input, open(output_file, 'r') as f_output:
        for _ in range(num_samples):
            input_line = f_input.readline().strip()
            output_line = f_output.readline().strip()

            # Check if either line is empty
            if not input_line or not output_line:
                continue  # Skip empty lines

            try:
                input_data = np.array(ast.literal_eval(input_line), dtype=np.float32)
                output_data = np.array(ast.literal_eval(output_line), dtype=np.float32)
            except SyntaxError:
                continue  # Skip lines causing syntax errors

            yield input_data, output_data

In [ ]:
def count_samples(file_path):
    with open(file_path, 'r') as f:
        return sum(1 for _ in f)
num_samples = count_samples('train_output_monthy_cleaned.txt')
num_samples

In [ ]:
num_samples = 166210
data_generator = read_data('train_input_monthy_cleaned.txt', 'train_output_monthy_cleaned.txt', num_samples)

inputs_monthy_3 = []
outputs_monthy_3 = []
for _ in range(num_samples):
    input_data, output_data = next(data_generator, (None, None))
    if input_data is not None and output_data is not None:
        inputs_monthy_3.append(input_data)
        outputs_monthy_3.append(output_data)

inputs_monthy_3 = torch.tensor(inputs_monthy_3, dtype=torch.float32)
outputs_monthy_3 = torch.tensor(outputs_monthy_3, dtype=torch.float32)

print(inputs_monthy_3.shape)
print(outputs_monthy_3.shape)

In [ ]:
# Create a TensorDataset from the inputs and outputs
dataset = TensorDataset(inputs_monthy_3, outputs_monthy_3)

# Define the batch size
batch_size = 512

# Create a DataLoader from the dataset
# Note: we set "shuffle=True" to shuffle the dataset before each epoch of training
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Print the number of batches
print(f'Number of batches: {len(data_loader)}')

In [ ]:
model = Quantformer(input_dim=2, hidden_dim=64, num_heads=8, num_layers=6, output_dim = 3)

# Define the loss function (Mean Squared Error Loss)
criterion = nn.MSELoss()

# Initialize the optimizer (Adam optimizer)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 50

# Move the model and the loss function to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

# Start training
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        # Get the inputs and outputs and move them to the GPU if available
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass: compute the outputs by passing inputs to the model
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass: compute the gradients by performing backpropagation
        loss.backward()

        # Update the parameters of the model
        optimizer.step()

        # Update the running loss
        running_loss += loss.item()

    # Print the average loss for this epoch
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss / len(data_loader)))


torch.save(model, 'trans_model_monthy_1.pth')
torch.save(model.state_dict(), 'trans_params_monthy_1.pth')

# Backtest base on the result

In [ ]:
def predict_list_at_date(df_input, model_params_path, date, rank, o_dim, frequency):
    # determine the date of the column
    if frequency == 'monthy':
        date_str = df_input.columns[date]  # eg. ("2020/01", "profit")
    else:
        column_n = df_input.columns[date]
        date_str = column_n.split('_')[0]

    # obtain the return df
    df_profit_rate = create_df_profit_rate_with_symbol(df_input, date_str, date, frequency)

    # Z-score
    df_profit_nor = normalize_dataframe(df_profit_rate)

    # get profit / turnover col, get the lastest 40 cols
    profit_turnoff_cols = [col for col in df_profit_nor.columns if 'profit' in col or 'turnoff' in col]
    profit_turnoff_cols = sorted(profit_turnoff_cols)[-40:]  

    normalized_df = df_profit_nor[profit_turnoff_cols]

    # build the [x,y] vector
    vector_list_per_row = normalized_df.apply(
        lambda row: [list(row[i:i+2]) for i in range(0, len(row), 2)],
        axis=1
    )

    data = []
    for symbol, vector in zip(df_profit_rate.index, vector_list_per_row):
        data.append([symbol, vector, None])

    df_date_rank = pd.DataFrame(data, columns=['symbol', 'vector', 'rank'])
    df_date_rank.set_index('symbol', inplace=True) 

    # get the prediction
    df_date_rank = back_test(model_params_path, df_date_rank, rank, o_dim)

    return df_date_rank


In [ ]:
def back_test(model_params_path, test_df, rank, o_dim):
    model = Quantformer(input_dim=2, hidden_dim=64, num_heads=8, num_layers=6, output_dim=o_dim)
    model.load_state_dict(torch.load(model_params_path))
    model.eval()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for i in tqdm(range(len(test_df['vector']))):
        new_input = test_df['vector'].iloc[i]
        new_input_tensor = torch.tensor(new_input, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(new_input_tensor)

        output = output.cpu().numpy()
        test_df.loc[test_df.index[i], 'rank'] = output[0][0]

    # sort by 'rank' 
    test_df_sorted = test_df.sort_values(by='rank', ascending=False)

    # get the col in top N
    num_selected = int(len(test_df_sorted) * (rank / 100))
    selected_indices = test_df_sorted.index[:num_selected]

    selected_df = test_df.loc[selected_indices].copy()

    selected_df.reset_index(drop=False, inplace=True) 

    return selected_df


Get predict stock list

In [ ]:
# the earliest backtest date 2020/01
date = 241 
while date < 242: 
# the latest backtest date 2024/12
 
    date_str = df_input_monthy.columns[date]
    desired_date_str = date_str[0]
    
    #get the predict stocklist at the date
    df_temp_daily = predict_list_at_date(df_input_monthy, 'trans_params_monthy.pth', date, 20, 3, 'monthy') 
    
    symbol_values_str = df_temp_daily['symbol'].astype(str).values
    desired_length = len(df_monthy_3)
    current_length = len(df_temp_daily['symbol'].values)
    difference = desired_length - current_length

    if difference > 0:
        padded_values = np.pad(symbol_values_str, (0, difference), constant_values='nan')
        df_monthy_3.loc[:, desired_date_str] = padded_values
    else:
        df_monthy_3.loc[:, desired_date_str] = symbol_values_str[:desired_length]
    
    # +2 as there are 2 cols
    date = date + 2
    print(desired_date_str)
    
#record in csv
df_monthy_3.to_csv('min_20_back_monthy.csv', index=False)

Calculate the profit

In [ ]:
file_path = 'trade_monthy.csv'
stock_list_data=pd.read_csv(file_path).dropna()

def convert_col_to_ym(col):
    try:
        date = pd.to_datetime(col, format='%Y/%m', errors='coerce')
        if pd.notna(date):
            return date.strftime('%Y-%m')
        else:
            return col
    except:
        return col
    
# replace the name of the column    
stock_list_data.columns = [convert_col_to_ym(col) for col in stock_list_data.columns]
stock_list_data

In [ ]:
def profit_in_strategy(start_day, end_day, stock_list, data_list):  
    #stock_list is the stock list hold on that month
    #data_list the list that save the profit of each month
    
    #create dataframe
    df_profit = {'profit':[]}
    date_range = pd.date_range(start = start_day, end = end_day, freq='M')
    df_profit = pd.DataFrame(columns=['profit'] + [date.strftime('%Y/%m') for date in date_range])
    profit_data = {}
    
    #change the formation of the time
    start_day = pd.to_datetime(start_day, format='%Y%m%d')
    end_day = pd.to_datetime(end_day, format='%Y%m%d')
    
    while start_day != end_day:
        profit_temp = 0
        column_name = start_day.strftime('%Y/%m')
        column_name_1 = start_day.strftime('%Y/%m')
        column_name_2 = (start_day.strftime('%Y-%m'), 'profit')
        stocklist = stock_list[column_name_1]
        for i in range(0,len(stocklist)):
            data = data_list.loc[data_list['symbol'] == stocklist[i]] #us stock index are str
            profit_stock = data[column_name_2].values[0]
            profit_temp = profit_stock + profit_temp
        
        #calculate profit
        profit_temp = profit_temp/len(stocklist)
        profit_data[column_name] = profit_temp
        #df_profit[column_name][0] = profit_temp
        print(start_day.strftime('%Y/%m'))
        print(profit_temp)
        #to next timestamp
        start_day = start_day + pd.DateOffset(months=1, day=1)
    
    df_profit = pd.DataFrame([profit_data])
    return df_profit

In [ ]:
stock_list_data = pd.read_csv("trade_monthy.csv")

In [ ]:
#get the return
profit_df = profit_in_strategy('20200101', '20241231', stock_list_data, df_input_m)

Plot

In [ ]:
dates = pd.date_range(start="2020-01-01", periods=len(returns), freq='M')
strategy_returns = pd.Series(returns, index=dates)

In [ ]:
returns = profit_df.iloc[0]

nasdaq_returns_percent = [
    1.23, -6.78, -11.16, 19.17, 9.31, 6.20, 6.77, 8.54, -5.77, -3.37,
    10.79, 4.67, 0.87, -0.26, -1.19, 4.09, -2.02, 4.88, 1.23, 3.39,
    -5.62, 6.92, -0.02, -0.68, -9.49, -3.43, 3.41, -13.26, -2.05, -8.71,
    12.35, -4.64, -10.50, 3.90, 4.37, -8.73, 10.68, -1.11, 6.69, 0.04,
    5.80, 6.59, 4.05, -2.17, -5.81, -2.78, 10.70, 5.52, 1.95, 5.49,
    1.67, -4.51, 6.96, 5.14, -0.98, 0.38, 3.43, -0.33, 5.65
]
# nasdaq return in the past months

dates = pd.date_range(start="2020-01-01", periods=len(returns), freq='M')
nasdaq_returns = pd.Series(nasdaq_returns_percent, index=dates)


returns.index = pd.to_datetime(returns.index, format="%Y/%m")  
returns = returns.sort_index() 

cumulative_returns = (1 + returns/100).cumprod()
nasdaq_cumulative = (1 + nasdaq_returns / 100).cumprod()


plt.figure(figsize=(12, 6))
plt.plot(cumulative_returns, marker='o', label='Quantformer')
plt.plot(nasdaq_cumulative, marker='x', label='NASDAQ Composite')
plt.title('Cumulative Return Comparison (2020-2024)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

final_cumulative_return = cumulative_returns.iloc[-1]
final_nasdaq_return = nasdaq_cumulative.iloc[-1] - 1
print("Final Cumulative Return:", final_cumulative_return-1)
print(f"Final Cumulative Return - NASDAQ Composite: {final_nasdaq_return:.2%}")


In [ ]:
#get the max drawdown (MDD)

# cumulative return
cumulative_returns = (1 + returns/100).cumprod()

# swap to the series
cumulative_returns = pd.Series(cumulative_returns.values, index=cumulative_returns.index)

# calculate to the highest point
rolling_max = cumulative_returns.cummax()

# calculate the line of drawdown
drawdown = (cumulative_returns - rolling_max) / rolling_max

# get the MDD
max_drawdown = drawdown.min()

print(f" Max drawdown: {max_drawdown:.2%}")


In [ ]:
# get 95% VaR

profit_series = profit_df.iloc[0]  

# 95% VaR
var_95 = profit_series.quantile(0.05)

print(f" 95% VaR: {var_95:.4f}")
